In [3]:
import pandas as pd
from functions import *

In [ ]:
data_tpts, data_x, data_y = data_deal('Sethylene%')
data_y = data_y / 100
index_z_2 = compared(n = 10, seed_z = 123, data_x = data_x, data_y = data_y, data_tpts = data_tpts, 
                   n_basis = 5, order = 4, test_size = 0.2, plot_flag=True,
                   method_list = ['lin_b','knn_b','gbdt_b','rsf_b','xgb_b','MBR_XGB'])
df = pd.DataFrame(index_z_2)
df.to_csv('...', index=False)

In [ ]:
data_tpts, data_x, data_y = data_deal_1('Spropylene%')
index_z_3 = compared(n = 10, seed_z = 123, data_x = data_x, data_y = data_y, data_tpts = data_tpts, 
                   n_basis = 5, order = 4, test_size = 0.2, plot_flag=True,
                   method_list = ['lin_b','knn_b','gbdt_b','rsf_b','xgb_b','MBR_XGB'])
df = pd.DataFrame(index_z_3)
df.to_csv('...', index=False)